In [1]:
import akshare as ak
import pandas as pd
import numpy as np
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
import datetime
import webbrowser
import os
import talib
from pyecharts.globals import CurrentConfig, NotebookType
#CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

def cal_K(some_arr):
    y_arr = np.array(some_arr).ravel()
    x_arr = list(range(1,len(y_arr)+1))
    fit_K = np.polyfit(x_arr,y_arr, deg=1)
    return round(fit_K[0],2)

def plot_kline_dkc(code, start_date, end_date, dkc=60):
    today = datetime.date.today()
    yesterday = today - datetime.timedelta(days=1)
    current_day = yesterday.strftime('%Y%m%d') 
    
    name_code =ak.stock_zh_a_spot_em()  
    name = name_code[name_code['代码'] == code]['名称'].values[0]

    df = ak.stock_zh_a_hist(symbol=code, start_date=start_date,end_date=end_date, adjust="qfq").iloc[:, :6]
    
    df.columns = ['date','open','close','high','low','volume',]
    df['volume'] = round(df['volume'].astype('float')/10000, 2)
    dkc_name = "dkc{}".format(dkc)
    gll_name = "gll{}".format(dkc)
    k_name = "kma{}".format(dkc)

    # 把date作为日期索引
    df.index = pd.to_datetime(df.date)
    df.index=df.index.strftime('%Y%m%d')
    df=df.sort_index()
    df['ma5']=df.close.rolling(5).mean()
    df['ma10']=df.close.rolling(10).mean()
    df['ma20']=df.close.rolling(20).mean()
    #df['ma30']=df.close.rolling(30).mean()
    df['ma60']=df.close.rolling(60).mean()
    df['ma120']=df.close.rolling(120).mean()
    df['ma250']=df.close.rolling(250).mean()

    df.index=range(len(df)) #修改索引为数字序号
    df['ATR1']=df['high']-df['low'] #当日最高价-最低价
    df['ATR2']=abs(df['close'].shift(1)-df['high']) #上一日收盘价-当日最高价
    df['ATR3']=abs(df['close'].shift(1)-df['low']) #上一日收盘价-当日最低价
    df['ATR4']=df['ATR1']
    for i in range(len(df)): #取价格波动的最大值
        if df.loc[i,'ATR4']<df.loc[i,'ATR2'] :
            df.loc[i,'ATR4']=df.loc[i,'ATR2']
        if df.loc[i,'ATR4']<df.loc[i,'ATR3'] :
            df.loc[i,'ATR4']=df.loc[i,'ATR3']
    df['ATR']=df.ATR4.rolling(14).mean() # N=14的ATR值
    df['stop']=df['close'].shift(1)-df['ATR']*3 #止损价=(上一日收盘价-3*ATR)

    # 计算抵扣差
    df['dkc5'] = df["close"]-df["close"].shift(4)
    df['dkc10'] = df["close"]-df["close"].shift(9)
    df['dkc20'] = df["close"]-df["close"].shift(19)
    #df['dkc30'] = df["close"]-df["close"].shift(29)
    df['dkc60'] = df["close"]-df["close"].shift(59)
    df['dkc120'] = df["close"]-df["close"].shift(119)
    df['dkc250'] = df["close"]-df["close"].shift(249)
    
    # 计算乖离率
    df['gll5'] = round((df["close"]-df["ma5"])*100/df["ma5"],2)
    df['gll10'] = round((df["close"]-df["ma10"])*100/df["ma10"],2)
    df['gll20'] = round((df["close"]-df["ma20"])*100/df["ma20"],2)
    #df['dkc30'] = df["close"]-df["lma30"]
    df['gll60'] = round((df["close"]-df["ma60"])*100/df["ma60"],2)
    df['gll120'] = round((df["close"]-df["ma120"])*100/df["ma120"],2)
    df['gll250'] = round((df["close"]-df["ma250"])*100/df["ma250"],2)
    
    # 计算k率
    df['kma5']=df.close.rolling(5).apply(cal_K)
    df['kma10']=df.close.rolling(10).apply(cal_K)
    df['kma20']=df.close.rolling(20).apply(cal_K)
    #df['klma30']=df.close.rolling(30).apply(cal_K)
    df['kma60']=df.close.rolling(60).apply(cal_K)
    df['kma120']=df.close.rolling(120).apply(cal_K)
    df['kma250']=df.close.rolling(250).apply(cal_K)

    #BOLL计算 取N=13，M=2
    df['boll']=df.close.rolling(13).mean()
    df['delta']=df.close-df.boll
    df['beta']=df.delta.rolling(13).std()
    df['up']=df['boll']+2*df['beta']
    df['down']=df['boll']-2*df['beta']

    #标记买入和卖出信号
    markb=pd.DataFrame()
    marks=pd.DataFrame()
    for i in range(len(df)):
        if df.loc[i,'close']>df.loc[i,'up'] :
            marks.loc[i,'date']=df.loc[i,'date']
            marks.loc[i,'close']=df.loc[i,'close']
            marks.loc[i,'act']='S'
        if df.loc[i,'close']<df.loc[i,'down'] :
            markb.loc[i,'date']=df.loc[i,'date']
            markb.loc[i,'close']=df.loc[i,'close']
            markb.loc[i,'act']='B'

    df.index = pd.to_datetime(df.date)
    df.index=df.index.strftime('%Y%m%d')

    kline = (
        Kline(init_opts=opts.InitOpts(width="1200px",height="600px"))
        .add_xaxis(xaxis_data=list(df.index)) #X轴数据
        .add_yaxis(
            series_name="klines", #序列名称
            y_axis=df[["open","close","low","high"]].values.tolist(), #Y轴数据
            itemstyle_opts=opts.ItemStyleOpts(color="#ec0000", color0="#00da3c"),
            markpoint_opts=opts.MarkPointOpts(
                data=[#添加标记符
                opts.MarkPointItem(type_='max', name='最大值', value_dim='highest'),
                opts.MarkPointItem(type_='min', name='最小值', value_dim='lowest'), ],
                #symbol='circle',
                #symbol_size=[100,30]
            ),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="{}-K线及均线".format(name),pos_left='45%'), #标题位置
            legend_opts=opts.LegendOpts(pos_right="15%",pos_top="5%"), #图例位置
            #legend_opts=opts.LegendOpts(is_show=True, pos_bottom=10, pos_left="center"),
            datazoom_opts=[
                opts.DataZoomOpts(
                    is_show=False,
                    type_="inside", #内部缩放
                    xaxis_index=[0,1,2],  #可缩放的x轴坐标编号
                    range_start=0, range_end=100, #初始显示范围
                ),
                opts.DataZoomOpts(
                    is_show=True, #显示滑块
                    type_="slider", #滑块缩放
                    xaxis_index=[0,1,2],  #可缩放的x轴坐标编号
                    pos_top="85%",
                    range_start=0, range_end=100, #初始显示范围
                ),
            ],
            yaxis_opts=opts.AxisOpts(
                is_scale=True, #缩放时是否显示0值
                splitarea_opts=opts.SplitAreaOpts( #分割显示设置
                    is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1) ),
            ),
            tooltip_opts=opts.TooltipOpts( #提示框配置
                trigger="axis", #坐标轴触发提示
                axis_pointer_type="cross", #鼠标变为十字准星
                background_color="rgba(245, 245, 245, 0.8)", #背景颜色
                border_width=1, border_color="#ccc", #提示框配置
                textstyle_opts=opts.TextStyleOpts(color="#000"), #文字配置
            ),
            visualmap_opts=opts.VisualMapOpts( #视觉映射配置
                is_show=False, dimension=2,
                series_index=5, is_piecewise=True,
                pieces=[ {"value": 1, "color": "#00da3c"}, {"value": -1, "color": "#ec0000"},             ],
            ),
            axispointer_opts=opts.AxisPointerOpts( #轴指示器配置
                is_show=True,
                link=[{"xAxisIndex": "all"}],
                label=opts.LabelOpts(background_color="#777"), #显示标签设置
            ),
            brush_opts=opts.BrushOpts(
                x_axis_index="all", #所有series
                brush_link="all", #不同系列选中后联动
                out_of_brush={"colorAlpha": 0.1}, #高亮显示程度
                brush_type="lineX", #纵向选择
            ),
        )
    )

    #均线
    line=Line()
    line.add_xaxis( df.index.tolist() ) #X轴数据
    line.add_yaxis( 'MA5', #序列名称
                    df.ma5.round(2).tolist(), #Y轴数据
                    is_smooth=True, #平滑曲线
                    is_symbol_show=False #不显示折线的小圆圈
    )
    line.add_yaxis( 'MA10',df.ma10.round(2).tolist(),is_smooth=True,is_symbol_show=False )
    line.add_yaxis( 'MA20',df.ma20.round(2).tolist(),is_smooth=True,is_symbol_show=False )
    #line.add_yaxis( 'MA30',df.lma30.round(2).tolist(),is_smooth=True,is_symbol_show=False )
    line.add_yaxis( 'MA60',df.ma60.round(2).tolist(),is_smooth=True,is_symbol_show=False )
    line.add_yaxis( 'MA120',df.ma120.round(2).tolist(),is_smooth=True,is_symbol_show=False )
    line.add_yaxis( 'MA250',df.ma250.round(2).tolist(),is_smooth=True,is_symbol_show=False )
    # BOLL线
    line.add_yaxis( 'boll',df.boll.round(2).tolist(),is_smooth=True,is_symbol_show=False,linestyle_opts=opts.LineStyleOpts(width=2,type_='dashed') )
    line.add_yaxis( 'up',df.up.round(2).tolist(),is_smooth=True,is_symbol_show=False,linestyle_opts=opts.LineStyleOpts(width=2,type_='dashed') )
    line.add_yaxis( 'down',df.down.round(2).tolist(),is_smooth=True,is_symbol_show=False,linestyle_opts=opts.LineStyleOpts(width=2,type_='dashed') )

    line.set_series_opts(
        label_opts=opts.LabelOpts(is_show=False), #是否显示数据标签
        linestyle_opts=opts.LineStyleOpts(width=1), #线宽
    )
    line.set_global_opts(
        datazoom_opts=[
            opts.DataZoomOpts(
                is_show=False,
                type_="inside", #图内缩放调整
                xaxis_index=[0,1],  #可缩放的x轴坐标编号
                range_start=0, range_end=100, #初始显示范围
            ),
            opts.DataZoomOpts(
                is_show=True, #是否显示滑块
                type_="slider", #外部滑块缩放调整
                xaxis_index=[0,1],  #可缩放的x轴坐标编号
                pos_top="85%",
                range_start=0, range_end=100, #初始显示范围
            ),
        ],
        legend_opts=opts.LegendOpts(pos_right="20%",pos_top="5%"), #图例位置
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross") #趋势线设置
    )
    stop=(
        Line()
        .add_xaxis( df.index.tolist() ) #X轴数据
        .add_yaxis( 'STOP', #序列名称
                    df.stop.round(2).tolist(), #Y轴数据
                    is_smooth=True, #平滑曲线
        )
        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False), #是否显示数据标签
            linestyle_opts=opts.LineStyleOpts(width=2), #线宽
        )
        .set_global_opts(
            datazoom_opts=[
                opts.DataZoomOpts(
                    is_show=False,
                    type_="inside", #图内缩放调整
                    xaxis_index=[0,1],  #可缩放的x轴坐标编号
                    range_start=0, range_end=100, #初始显示范围
                ),
                opts.DataZoomOpts(
                    is_show=True, #是否显示滑块
                    type_="slider", #外部滑块缩放调整
                    xaxis_index=[0,1],  #可缩放的x轴坐标编号
                    pos_top="85%",
                    range_start=0, range_end=100, #初始显示范围
                ),
            ],
            legend_opts=opts.LegendOpts(pos_right="20%",pos_top="5%"), #图例位置
            tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross") #趋势线设置
        )
    )
    #买卖标记点
    markb.index = pd.to_datetime(markb.date)
    markb.index=markb.index.strftime('%Y%m%d')
    mark1 = ( #买入信号
        Scatter()
        .add_xaxis(markb.index.tolist())
        .add_yaxis(series_name='',
                   y_axis=markb.close.tolist(),
                   xaxis_index=0,
                   symbol='triangle',
                   symbol_size=10,#设置散点的大小
                  )
        .set_series_opts(label_opts = opts.LabelOpts(is_show =False),
                         itemstyle_opts = opts.ItemStyleOpts(color="red"))
        .set_global_opts(legend_opts=opts.LegendOpts(is_show=False))
        .set_global_opts(visualmap_opts=opts.VisualMapOpts(is_show=False))
    )
    marks.index = pd.to_datetime(marks.date)
    marks.index=marks.index.strftime('%Y%m%d')
    mark2 = ( #卖出信号
        Scatter()
        .add_xaxis(marks.index.tolist())
        .add_yaxis(series_name='',
                   y_axis=marks.close.tolist(),
                   xaxis_index=0,
                   symbol='triangle',
                   symbol_size=10,#设置散点的大小
                   symbol_rotate=180,
                  )
        .set_series_opts(label_opts = opts.LabelOpts(is_show =False),
                         itemstyle_opts = opts.ItemStyleOpts(color="green"))
        .set_global_opts(legend_opts=opts.LegendOpts(is_show=False))
        .set_global_opts(visualmap_opts=opts.VisualMapOpts(is_show=False))
    )
    kline.overlap(line)
    kline.overlap(stop) #止损数据叠加到K线
    kline.overlap(mark1)
    kline.overlap(mark2)


    #成交量
    bar = (
        Bar()
        .add_xaxis(xaxis_data=list(df.index)) #X轴数据
        .add_yaxis(
            series_name="成交量(百万)",
            y_axis=df["volume"].tolist(), #Y轴数据
            xaxis_index=1,
            yaxis_index=1,
            label_opts=opts.LabelOpts(is_show=False),
            itemstyle_opts=opts.ItemStyleOpts(
                    color=JsCode(
                        """
                    function(params) {
                        var colorList;
                        if (barData[params.dataIndex][1] > barData[params.dataIndex][0]) {
                            colorList = '#ef232a';
                        } else {
                            colorList = '#14b143';
                        }
                        return colorList;
                    }
                    """
                    )
                ),
        )
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                type_="category", #坐标轴类型-离散数据
                grid_index=1,
                axislabel_opts=opts.LabelOpts(is_show=False),
            ),
            legend_opts=opts.LegendOpts(is_show=False),
        )
    )
    #抵扣差
    line_dkc = (
        Line()
        .add_xaxis(xaxis_data=list(df.index)) #X轴数据
        .add_yaxis(
            series_name="抵扣差",
            y_axis=df[dkc_name].tolist(), #Y轴数据
            xaxis_index=1,
            yaxis_index=1,
            label_opts=opts.LabelOpts(is_show=False),
            itemstyle_opts=opts.ItemStyleOpts(
                color='#ef232a' #'#14b143'
            ),
            markline_opts=opts.MarkLineOpts(
                data=[opts.MarkLineItem(name='0值', y = 0, symbol='none', )], 
                linestyle_opts=opts.LineStyleOpts(width = 0.1, color = '#301934',),
            )
        )
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                type_="category", #坐标轴类型-离散数据
                grid_index=1,
                axislabel_opts=opts.LabelOpts(is_show=False),
            ),
            legend_opts=opts.LegendOpts(is_show=False),
        )
    )
    #乖离率
    line_gll = (
        Line()
        .add_xaxis(xaxis_data=list(df.index)) #X轴数据
        .add_yaxis(
            series_name="乖离率",
            y_axis=df[gll_name].tolist(), #Y轴数据
            xaxis_index=1,
            yaxis_index=1,
            label_opts=opts.LabelOpts(is_show=False),
            itemstyle_opts=opts.ItemStyleOpts(
                color='#ef232a' #'#14b143'
            ),
            markline_opts=opts.MarkLineOpts(
                data=[opts.MarkLineItem(name='0值', y = 0, symbol='none', )], 
                linestyle_opts=opts.LineStyleOpts(width = 0.1, color = '#301934',),
            )
        )
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                type_="category", #坐标轴类型-离散数据
                grid_index=1,
                axislabel_opts=opts.LabelOpts(is_show=False),
            ),
            legend_opts=opts.LegendOpts(is_show=False),
        )
    )
    #k率
    line_k = (
        Line()
        .add_xaxis(xaxis_data=list(df.index)) #X轴数据
        .add_yaxis(
            series_name="k率",
            y_axis=df[k_name].tolist(), #Y轴数据
            xaxis_index=1,
            yaxis_index=1,
            label_opts=opts.LabelOpts(is_show=False),
            itemstyle_opts=opts.ItemStyleOpts(
                color='#ef232a' #'#14b143'
            ),
            markline_opts=opts.MarkLineOpts(
                data=[opts.MarkLineItem(name='0值', y = 0, symbol='none', )], 
                linestyle_opts=opts.LineStyleOpts(width = 0.1, color = '#301934',),
            )
        )
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                type_="category", #坐标轴类型-离散数据
                grid_index=1,
                axislabel_opts=opts.LabelOpts(is_show=False),
            ),
            legend_opts=opts.LegendOpts(is_show=False),
        )
    )

    #图像排列
    grid_chart = Grid(
        init_opts=opts.InitOpts(
            width="1200px", #显示图形宽度
            height="600px",
            animation_opts=opts.AnimationOpts(animation=False), #关闭动画
        )
    )
    grid_chart.add_js_funcs("var barData={}".format(df[["open","close"]].values.tolist())) # 导入open、close数据到barData改变交易量每个bar的颜色

    grid_chart.add( #加入均线图
        kline,
        grid_opts=opts.GridOpts(pos_left="10%", pos_right="8%", height="40%"),
    )
    grid_chart.add( #加入成交量图
        bar,
        grid_opts=opts.GridOpts(pos_left="10%", pos_right="8%", pos_top="60%", height="20%"),
    )
    grid_chart.add( #加入抵扣差
#         line_dkc,
#         line_gll,
        line_k,
        grid_opts=opts.GridOpts(pos_left="10%", pos_right="8%", pos_top="80%", height="20%"),
    )
        
    grid_chart.render("kline_volume_dkc_{}.html".format(name))
    webbrowser.open_new_tab('file://' + os.path.realpath('kline_volume_dkc_{}.html').format(name))

D:\Apps\Scoop\apps\anaconda3-2022.05\2022.05\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
D:\Apps\Scoop\apps\anaconda3-2022.05\2022.05\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
plot_kline_dkc("002230", '20200101', '20240314', 20)

In [16]:
plot_kline_dkc("600733", '20200101', '20240314')

In [17]:
plot_kline_dkc("002261", '20200101', '20240314')

In [18]:
plot_kline_dkc("002771", '20200101', '20240314')

In [19]:
plot_kline_dkc("603496", '20200101', '20240314')

In [12]:
plot_kline_dkc("000063", '20200101', '20240314')